<a href="https://colab.research.google.com/github/ngoan22mse23088/python4engineers-assignments/blob/master/Assignments_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Môn học : Python For Engineers - Assignment 3

Dựa trên templates và dữ liệu được cung cấp. 
Viết CRUD database function thông qua api với Flask. Hiển thị kết quả trên cửa sổ thực thi và templates.
Templates : https://drive.google.com/file/d/1LUZuEqw2NiRaiQ2sVrDkFt5r4H-3UXgR/view

In [ ]:
# cài đặt packages version như sau
!pip install opendatasets
!pip install mysql-connector-python
!pip install pandas matplotlib
!apt-get update install mysql-server mysql-client -y
!service mysql start

# cài đặt môi trường mysql trên colab với thông số :
# host : localhost
# user : root
# pass : root
# database : mydatabase
!mysql -u root -p -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root';"
!mysql -u root -p -e "CREATE DATABASE mydatabase;"
!mysql -u root -p -e "SHOW DATABASES;"
!mysql -u root -p -e "GRANT ALL PRIVILEGES ON mydatabase.* TO 'root'@'localhost';"

In [ ]:
# Thêm các thư viện cơ bản để xử lý
from flask import Flask, flash, render_template, request
import mysql.connector
import pymysql

app = Flask(__name__)
app.secret_key = "assignment3"

class Database:
    # Hàm kết nối
    def connect(self):
        return pymysql.connect(host="localhost", user="root", password="12345678", database="mydatabase", charset='utf8mb4')

    # Hàm đọc data table
    def read(self, id):
        con = Database.connect(self)
        cursor = con.cursor()

        try:
            if id == None:
                cursor.execute("SELECT * FROM posts order by id asc")
            else:
                cursor.execute(
                    "SELECT * FROM posts where id = %s order by id asc", (id))

            return cursor.fetchall()
        except:
            return ()
        finally:
            con.close()

    # Hàm get data từ table posts
    def get_posts(self):
        con = Database.connect(self)
        cursor = con.cursor()

        try:
            cursor.execute("SELECT id, title, content FROM posts")
            posts = cursor.fetchall()
            return [Post(*post) for post in posts]
        except:
            return ()
        finally:
            con.close()
        pass

    # Hàm get post theo id
    def get_post_by_id(self, id):
        con = Database.connect(self)
        cursor = con.cursor()

        try:
            cursor.execute(
                "SELECT id, title, content FROM posts WHERE id = %s", (id))
            post = cursor.fetchone()
            return Post(post[0], post[1], post[2])
        except:
            return None
        finally:
            con.close()
        pass

    # Hàm thêm mới 1 post
    def insert(self, post):
        con = Database.connect(self)
        cursor = con.cursor()

        try:
            cursor.execute("INSERT INTO posts(title, content) VALUES(%s, %s)",
                           (post.title, post.content))
            con.commit()

            return True
        except:
            con.rollback()

            return False
        finally:
            con.close()

    # Hàm cập nhật 1 post
    def update(self, post):
        con = Database.connect(self)
        cursor = con.cursor()

        try:
            cursor.execute("UPDATE posts set title = %s, content = %s where id = %s",
                           (post.title, post.content, post.id))
            con.commit()

            return True
        except:
            con.rollback()

            return False
        finally:
            con.close()

    # Hàm xóa 1 post theo id
    def delete(self, id):
        con = Database.connect(self)
        cursor = con.cursor()

        try:
            cursor.execute("DELETE FROM posts where id = %s", (id,))
            con.commit()

            return True
        except:
            con.rollback()

            return False
        finally:
            con.close()

class Post:
    def __init__(self, id, title, content):
        self.id = id
        self.title = title
        self.content = content

db = Database()

# Kết nối tới MySQL (port mặc định 3306)
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="mydatabase"
)

# Tạo table nếu chưa có
mycursor = mydb.cursor()
mycursor.execute(
    "CREATE TABLE IF NOT EXISTS posts (id int NOT NULL AUTO_INCREMENT,title varchar(255),content varchar(255),PRIMARY KEY (id));")
mycursor.execute("SELECT * FROM posts")
myresult = mycursor.fetchall()

# Trang main
@app.route('/')
def index():
    return render_template('index.html')

# Trang create
@app.route('/create', methods=['GET'])
def create():
    return render_template('create.html')

# Trang create
@app.route('/create', methods=['POST'])
def addPost():
    form = request.form
    title = form.get('title')
    content = form.get('content')
    post = Post(None, title, content)
    if db.insert(post):
        flash("Post has been added")
    else:
        flash("Post can not be added")
    return posts()

# Trang lấy 1 post theo id
@app.route('/edit/<int:id>/', methods=['GET'])
def edit(id):
    post = db.get_post_by_id(id)
    return render_template('edit.html', post=post)

# Trang cập nhât 1 post theo id
@app.route('/edit/<int:id>/', methods=['POST'])
def update_edit(id):
    form = request.form
    title = form.get('title')
    content = form.get('content')
    post = Post(id, title, content)
    if db.update(post):
        flash("A post has been updated")
    else:
        flash("A post can not be updated")
    post = db.get_post_by_id(id)
    return render_template('edit.html', post=post)

# Trang get ds post theo id
@app.route('/posts', methods=['GET'])
def posts():
    posts = db.get_posts()
    return render_template('posts.html', posts=posts)

# Trang xóa 1 post theo id
@app.route('/delete/<int:id>', methods=['POST'])
def delete(id):
    if db.delete(id):
        flash("A post has been deleted")
    else:
        flash("A post can not be deleted")
    return posts()

# Trang xử lý khi not found
@app.errorhandler(404)
def page_not_found(error):
    return render_template('error.html')

if __name__ == '__main__':
    app.run(port=8181, host="0.0.0.0")
